# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc19072e520>
├── 'a' --> tensor([[ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084]])
└── 'x' --> <FastTreeValue 0x7fc19072e850>
    └── 'c' --> tensor([[ 0.3534, -0.1203, -1.4135, -0.4582],
                        [-0.4694,  0.4467,  0.5777,  1.3706],
                        [-0.1595, -0.5754,  0.5351, -0.5888]])

In [4]:
t.a

tensor([[ 0.1573,  2.0848, -0.2895],
        [-1.6838,  1.2582, -0.9084]])

In [5]:
%timeit t.a

79 ns ± 2.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc19072e520>
├── 'a' --> tensor([[-0.5969,  0.3443,  1.4018],
│                   [ 0.5940, -0.7544,  2.1891]])
└── 'x' --> <FastTreeValue 0x7fc19072e850>
    └── 'c' --> tensor([[ 0.3534, -0.1203, -1.4135, -0.4582],
                        [-0.4694,  0.4467,  0.5777,  1.3706],
                        [-0.1595, -0.5754,  0.5351, -0.5888]])

In [7]:
%timeit t.a = new_value

79 ns ± 0.874 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084]]),
    x: Batch(
           c: tensor([[ 0.3534, -0.1203, -1.4135, -0.4582],
                      [-0.4694,  0.4467,  0.5777,  1.3706],
                      [-0.1595, -0.5754,  0.5351, -0.5888]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1573,  2.0848, -0.2895],
        [-1.6838,  1.2582, -0.9084]])

In [11]:
%timeit b.a

68.2 ns ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1240, -0.5109, -1.2257],
               [-0.0453,  1.8571,  2.4314]]),
    x: Batch(
           c: tensor([[ 0.3534, -0.1203, -1.4135, -0.4582],
                      [-0.4694,  0.4467,  0.5777,  1.3706],
                      [-0.1595, -0.5754,  0.5351, -0.5888]]),
       ),
)

In [13]:
%timeit b.a = new_value

662 ns ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

296 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

287 µs ± 9.29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc164bcabb0>
├── 'a' --> tensor([[[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]],
│           
│                   [[ 0.1573,  2.0848, -0.2895],
│                    [-1.6838,  1.2582, -0.9084]]])
└── 'x' --> <FastTreeValue 0x7fc164bca3a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.1 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc0e87b0a00>
├── 'a' --> tensor([[ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084],
│                   [ 0.1573,  2.0848, -0.2895],
│                   [-1.6838,  1.2582, -0.9084]])
└── 'x' --> <FastTreeValue 0x7fc164bcae20>
    └── 'c' --> tensor([[ 0.3534, -0.1203, -1.4135, -0.4582],
                        [-0.4694,  0.4467,  0.5777,  1.3706],
                 

In [23]:
%timeit t_cat(trees)

42.7 µs ± 997 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

81 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]],
       
               [[ 0.1573,  2.0848, -0.2895],
                [-1.6838,  1.2582, -0.9084]]]),
    x: Batch(
           c: tensor([[[ 0.3534, -0.1203, -1.4135, -0.4582],
                       [-0.4694,  0.4467,  0.5777,  1.3706],
                       [-0.1595, -0.5754,  0.5351, -0.5888]],
         

In [26]:
%timeit Batch.stack(batches)

106 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084],
               [ 0.1573,  2.0848, -0.2895],
               [-1.6838,  1.2582, -0.9084]]),
    x: Batch(
           c: tensor([[ 0.3534, -0.1203, -1.4135, -0.4582],
                      [-0.4694,  0.4467,  0.5777,  1.3706],
                      [-0.1595, -0.5754,  0.5351, -0.5888],
                      [ 0.3534, -0.1203, -1.4135, -0.4582],
                      [-0.4694,  

In [28]:
%timeit Batch.cat(batches)

191 µs ± 4.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

596 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
